In [1]:
with Dataset(self.path, mode='r') as f:
    for var in f.variables:
        # Station_ID and Observation_Class variables need
        # to be converted from byte string to string
        if var in ['Station_ID', 'Observation_Class']:
            data = f.variables[var][:]
            data = [i.tobytes(fill_value='/////', order='C')
                    for i in data]
            data = np.array(
                [''.join(i.decode('UTF-8', 'ignore').split())
                 for i in data])
            df_dict[var] = data

        # Grab variables with only 'nobs' dimension
        elif len(f.variables[var].shape) == 1:
            df_dict[var] = f.variables[var][:]

    # Create pandas dataframe from dict
    df = pd.DataFrame(df_dict)

    # Creates multidimensional indexed dataframe
    indices = ['Station_ID', 'Observation_Class', 'Observation_Type',
           'Observation_Subtype', 'Pressure', 'Height',
           'Analysis_Use_Flag']
    df.set_index(indices, inplace=True)

    # Rename columns
    df.columns = df.columns.str.lower()
    if self.variable == 'uv':
    for wind_type in ['u', 'v']:
        for bias_type in ['unadjusted', 'adjusted']:
            df = df.rename(columns={
                f'{wind_type}_obs_minus_forecast_{bias_type}':
                f'{wind_type}_omf_{bias_type}'
                })
            # Create hofx columns
            df[f'{wind_type}_hofx_{bias_type}'] = \
                df[f'{wind_type}_observation'] - \
                df[f'{wind_type}_omf_{bias_type}']

    else:
    for bias_type in ['unadjusted', 'adjusted']:
        df = df.rename(columns={
            f'obs_minus_forecast_{bias_type}': f'omf_{bias_type}',
            })
        # Create hofx columns
        df[f'hofx_{bias_type}'] = df['observation'] - \
            df[f'omf_{bias_type}']

    # Get index values
    self.obs_ids = df.index.get_level_values(
    'Observation_Type').unique().to_numpy()
    self.stn_ids = df.index.get_level_values(
    'Station_ID').unique().to_numpy()

    self.data_df = df

IndentationError: expected an indented block (<ipython-input-1-40bb301af34a>, line 30)

In [2]:
import xarray as xr
import glob

path = '/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/'

files = glob.glob(path + '*.nc4')
files.sort()

In [3]:
files

['/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_abi_g16_anl.2020092200.nc4',
 '/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_abi_g16_ges.2020092200.nc4',
 '/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_abi_g17_anl.2020092200.nc4',
 '/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_abi_g17_ges.2020092200.nc4',
 '/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_ahi_himawari8_anl.2020092200.nc4',
 '/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_ahi_himawari8_ges.2020092200.nc4',
 '/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_airs_aqua_anl.2020092200.nc4',
 '/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_airs_aqua_ges.2020092200.nc4',
 '/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_amsua_aqua_anl.2020092200.nc4',
 '/scratch2/NCEPDE

In [24]:
file = '/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_conv_t_ges.2020092200.nc4'

bad_vars = ('Bias_Correction_Terms')

ds = xr.open_dataset(file, drop_variables=bad_vars)
df = ds.to_dataframe()
df['Station_ID'] = df['Station_ID'].str.decode("utf-8")
df['Observation_Class'] = df['Observation_Class'].str.decode("utf-8")

# Creates multidimensional indexed dataframe
indices = ['Station_ID', 'Observation_Class', 'Observation_Type',
       'Observation_Subtype', 'Pressure', 'Height',
       'Analysis_Use_Flag']
df.set_index(indices, inplace=True)

df


Latitude  \
Station_ID Observation_Class Observation_Type Observation_Subtype Pressure    Height Analysis_Use_Flag              
ZSNO             t           180              1                   1031.599976 0.0     1.0              -35.000000   
VRWR7            t           180              1                   1030.000000 0.0     1.0              -35.200001   
                                                                                      1.0              -35.200001   
64501            t           181              0                   1013.299988 4.0    -1.0               -0.700000   
FOOG             t           187              0                   1013.599976 3.0    -1.0               -0.700000   
...                                                                                                           ...   
/////            t           180              0                   995.700012  0.0     1.0               80.629997   
                                                                  996.599976  0.0     1.0               80.610001   
                                                                  995.200012  0.0     1.0               80.639999   
                                                                  997.799988  0.0     1.0               80.599998   
                                                                  996.099976  0.0     1.0               80.620003   

                                                                                                         Longitude  \
Station_ID Observation_Class Observation_Type Observation_Subtype Pressure    Height Analysis_Use_Flag               
ZSNO             t           180              1                   1031.599976 0.0     1.0                 6.800000   
VRWR7            t           180              1                   1030.000000 0.0     1.0                 8.900000   
                                                                                      1.0                 9.000000   
64501            t           181              0                   1013.299988 4.0    -1.0                 8.750000   
FOOG             t           187              0                   1013.599976 3.0    -1.0                 8.750000   
...                                                                                                            ...   
/////            t           180              0                   995.700012  0.0     1.0               357.359985   
                                                                  996.599976  0.0     1.0               357.309998   
                                                                  995.200012  0.0     1.0               357.380005   
                                                                  997.799988  0.0     1.0               357.209991   
                                                                  996.099976  0.0     1.0               357.339996   

                                                                                                        Station_Elevation  \
Station_ID Observation_Class Observation_Type Observation_Subtype Pressure    Height Analysis_Use_Flag                      
ZSNO             t           180              1                   1031.599976 0.0     1.0                             0.0   
VRWR7            t           180              1                   1030.000000 0.0     1.0                             0.0   
                                                                                      1.0                             0.0   
64501            t           181              0                   1013.299988 4.0    -1.0                             4.0   
FOOG             t           187              0                   1013.599976 3.0    -1.0                             3.0   
...                                                                                                                   ...   
/////            t           180              0 

In [23]:
df

,Station_ID,Observation_Class,Observation_Type,Observation_Subtype,Latitude,Longitude,Station_Elevation,Pressure,Height,Time,...,Analysis_Use_Flag,Nonlinear_QC_Rel_Wgt,Errinv_Input,Errinv_Adjust,Errinv_Final,Observation,Obs_Minus_Forecast_adjusted,Obs_Minus_Forecast_unadjusted,Data_Pof,Data_Vertical_Velocity
nobs,,,,,,,,,,,,,,,,,,,,,
0,ZSNO,t,180,1,-35.000000,6.800000,0.0,1031.599976,0.0,0.00000,...,1.0,4.0,4.000000e-01,4.000000e-01,1.339194e-01,289.750000,1.425495,1.425495,1.000000e+09,1.000000e+09
1,VRWR7,t,180,1,-35.200001,8.900000,0.0,1030.000000,0.0,-3.00000,...,1.0,4.0,4.000000e-01,4.000000e-01,3.102377e-01,288.450012,0.605408,0.605408,1.000000e+09,1.000000e+09
2,VRWR7,t,180,1,-35.200001,9.000000,0.0,1030.000000,0.0,-2.00000,...,1.0,4.0,4.000000e-01,4.000000e-01,3.097822e-01,288.450012,0.583633,0.583633,1.000000e+09,1.000000e+09
3,64501,t,181,0,-0.700000,8.750000,4.0,1013.299988,4.0,-3.00000,...,-1.0,4.0,1.000000e-11,1.000000e-11,1.000000e-11,301.750000,1.256365,1.256365,1.000000e+09,1.000000e+09
4,FOOG,t,187,0,-0.700000,8.750000,3.0,1013.599976,3.0,-1.00000,...,-1.0,4.0,1.000000e-11,1.000000e-11,1.000000e-11,301.350006,1.052835,1.052835,1.000000e+09,1.000000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206866,/////,t,180,0,80.629997,357.359985,0.0,995.700012,0.0,-1.91667,...,1.0,4.0,4.000000e-01,4.000000e-01,3.128819e-01,272.350006,-0.186130,-0.186130,1.000000e+09,1.000000e+09
206867,/////,t,180,0,80.610001,357.309998,0.0,996.599976,0.0,0.08333,...,1.0,4.0,4.000000e-01,4.000000e-01,3.130049e-01,271.950012,-0.301150,-0.301150,1.000000e+09,1.000000e+09
206868,/////,t,180,0,80.639999,357.380005,0.0,995.200012,0.0,-2.91667,...,1.0,4.0,4.000000e-01,4.000000e-01,3.127697e-01,272.450012,-0.197209,-0.197209,1.000000e+09,1.000000e+09


In [18]:
from netCDF4 import Dataset

with Dataset(file, 'r') as nc:
    print(nc)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Number_of_Predictors: 3
    date_time: 2020092200
    dimensions(sizes): nobs(206871), Station_ID_maxstrlen(8), Observation_Class_maxstrlen(7), Bias_Correction_Terms_arr_dim(3)
    variables(dimensions): |S1 Station_ID(nobs,Station_ID_maxstrlen), |S1 Observation_Class(nobs,Observation_Class_maxstrlen), int32 Observation_Type(nobs), int32 Observation_Subtype(nobs), float32 Latitude(nobs), float32 Longitude(nobs), float32 Station_Elevation(nobs), float32 Pressure(nobs), float32 Height(nobs), float32 Time(nobs), float32 Prep_QC_Mark(nobs), float32 Setup_QC_Mark(nobs), float32 Prep_Use_Flag(nobs), float32 Analysis_Use_Flag(nobs), float32 Nonlinear_QC_Rel_Wgt(nobs), float32 Errinv_Input(nobs), float32 Errinv_Adjust(nobs), float32 Errinv_Final(nobs), float32 Observation(nobs), float32 Obs_Minus_Forecast_adjusted(nobs), float32 Obs_Minus_Forecast_unadjusted(nobs), float32 Data_Pof(nobs), float32 Data_Ver